In [ ]:
import json  
from bs4 import BeautifulSoup, Tag, NavigableString
from urllib.parse import urljoin
import pandas as pd
from tqdm import tqdm 
import json


with open('courts.json') as f:
    data = json.load(f)


driver = webdriver.Chrome('chromedriver.exe')


def get_tables(link):
    try:
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html)
        content = soup.select_one("#content")
        tables = pd.read_html(content.prettify())
        tables_pool.append(tables)
        driver.close()
    except Exception as e:
        tables_pool.append(str(e))

    
def flatten(xss):
    return [x for xs in xss for x in xs]


dict_of_lists = {}

from tqdm import tqdm 
for i in tqdm(range(len(data[:3]))):
    try:
        base  = data[i]['site']
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(base)
        href = driver.find_element_by_xpath('//a[contains(text(), "СУДЕБНОЕ ДЕЛОПРОИЗВОДСТВО")]').get_attribute('href')
        driver.get(href)
        WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "tablcont")))


        html = driver.page_source
        soup = BeautifulSoup(html)
        table = soup.select("tablcont")
        
        tables_pool = []
        
        
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_tables, links)
            
        dict_of_lists[i] = (tables_pool, data[i]['region'])
    except Exception as e:
         dict_of_lists[i] = (str(e), data[i]['region'])